In [1]:
import pandas as pd
import numpy as np
import math
from scipy.signal import savgol_filter

import plotly.express as px

## Part 1 - Data Import
Datum about hospital occupancy and tansportation modes in the Netherlands and Poland are imported and organized.
- Original global hospital occupancy data including country, date and hospital occupancy. Obtained from OWD: `https://ourworldindata.org/covid-hospitalizations`

- Original Netherlands tranport modes data including travel modes, region, date, population, periods. Obtained from CBS: `https://opendata.cbs.nl/statline/portal.html?_la=en&_catalog=CBS`

- Original Poland tranpsot modes data including travel modes and year. Obtained from SP: \
    `https://stat.gov.pl/en/search/search.html`

### 1.1 Import covid data for this sub-question
- Firstly, import global covid data: the daily new confirmed cases per 1 million people.
- Extract data of the Netherlands and Poland from the global data.
- Drop unwanted coulumns.

In [2]:
# import the csv file contians data of global daily Daily new confirmed cases per million people  
file_path_h = './data/travelling modes/raw/Daily new confirmed cases per 1M.csv'
df = pd.read_csv(file_path_h, delimiter=',')

# drop unwanted rows
df_covid = df.loc[((df['location'] == 'Netherlands') | (df['location'] == 'Poland'))]

# drop unwanted columns
df_covid = df_covid[['location', 'date', 'new_cases', 'new_cases_per_million', 'new_deaths', 'new_deaths_per_million']]
df_covid.set_index('date', inplace = True)

df_covid.head(5)

,location,new_cases,new_cases_per_million,new_deaths,new_deaths_per_million
date,,,,,
2020-02-27,Netherlands,1.0,0.057,NaN,NaN
2020-02-28,Netherlands,5.0,0.285,NaN,NaN
2020-02-29,Netherlands,4.0,0.228,NaN,NaN
2020-03-01,Netherlands,8.0,0.455,NaN,NaN
2020-03-02,Netherlands,6.0,0.342,NaN,NaN


### 1.2 Import the transportation data
- Firstly, import the Netherlands transportation data.

In [3]:
# import tansport data of the Netherlands 
file_path = './data/travelling modes/raw/NL_O_MODES.csv'
df_transportNL = pd.read_csv(file_path, delimiter=';')

# drop unwanted colums 
df_transportNL = df_transportNL.drop(['ID', 'Margins', 'Population'], axis=1)
columns_name = {'ModesOfTravel':'Travel Modes', 'TripCharacteristics':'Trip Characteristics', 
                'RegionCharacteristics':'Region', 'Periods':'Year', 'Trips_1': 'Average Trip Per Day',
                'Trips_4': 'Average Trip Per Year'}
df_transportNL.rename(columns=columns_name, inplace=True)

# display data
df_transportNL.head(5)

,Travel Modes,Trip Characteristics,Region,Year,Average Trip Per Day,Average Trip Per Year
0,T001093,2031090,NL01,2018JJ00,2.06,107.0
1,T001093,2031090,NL01,2019JJ00,1.94,101.0
2,T001093,2031090,NL01,2020JJ00,1.64,85.0
3,T001093,2031090,NL01,2021JJ00,1.83,95.0
4,T001093,2031090,LD01,2018JJ00,1.97,103.0


- Secondly, import Poland transportation data.

In [4]:
# import transport data of Poland
file_path = './data/travelling modes/raw/PL_MODES.csv'
df_transportPL = pd.read_csv(file_path, delimiter=';')

# display data
df_transportPL

,Travel Modes,Region,percentage_2021,percentage_2020,percentage_2019,percentage_2018
0,Total,Poland,100.0,100.0,100.0,100.0
1,Railway transport,Poland,48.8,55.8,57.9,46.7
2,Road transport,Poland,47.6,42.7,39.9,50.8
3,Air transport,Poland,3.2,1.0,1.6,2.1
4,Inland waterway transport,Poland,0.2,0.2,0.2,0.2
5,Maritime transport,Poland,0.2,0.3,0.4,0.2


## Part 2 Data process

### 2.1 Covid data processing

#### 2.1.1 Check original covid data

In [5]:
# display daily new_cases in the Netherlands and Poland 
fig = px.line(df_covid, y='new_cases', color='location', title='Daily new cases')
fig.show()

- It is noticable that there are many noise in the covid data, to better analyse the relationship between covid and the change of transportation, the original covid data needs to be smmothed.

- Before smoothing the original covid data, missing values in the data file needs to be filled. The filling should only evaluate cells where there is a 'nan' and ignore existing values. The following section deals with this problem. 


#### 2.1.2 Define a data filling class

- Firstly, define a class to fill the missing value in the covid data frame.

- Secondly, each missing value is the weighted mean of the existing values, and the weight of each existing value is determined by the distance (difference of dates) between each existing point and to-be-filled point. The weight is calculated by the following formula:

 $weight = e^{(-dist^2/(2 * sigma^2))}$



In [6]:
class Datafilling:
    def __init__(self, column_1, column_2, mode='integer'):
        
        # split the columns by country
        self.dates_NL, self.dates_PL, self.values_NL, self.values_PL = self.column_split(column_1, column_2)

        # seperately deal with each half-column
        self.filled_sublist_1 = self.datafill(self.dates_NL, self.values_NL)
        self.filled_sublist_2 = self.datafill(self.dates_PL, self.values_PL)

        # combine two sub-filled list
        filled_list = [*self.filled_sublist_1, *self.filled_sublist_2]

        # format the result
        if mode =='integer':
            self.filled_column = ['%.1f' % member for member in filled_list]
        elif mode == 'float':
            self.filled_column = ['%.3f' % member for member in filled_list]


    def column_split(self, list_1, list_2):
        """
        split the column into two parts by countries
        """
        # seperate the column from the split point
        for i in range(1, len(list_1)-1):
            if list_1[i] > list_1[i+1]:
                dates_NL = list_1[:i+1]
                dates_PL = list_1[i+1:]
                values_NL = list_2[:i+1]
                values_PL = list_2[i+1:]
        
        return dates_NL, dates_PL, values_NL, values_PL


    def datafill(self, dateColumn, valueColumn):
            """ 
            fill missing values
            """
            # generate value container
            dates = np.linspace(0, len(dateColumn)-1, len(dateColumn))
            values = np.zeros(len(dateColumn))

            # compute values 
            sigma = np.nanstd(valueColumn)
            for i in range(len(dates)):
                value = self.compute_value(dates, valueColumn, i, sigma)
                values[i] = value
            
            return values


    def compute_value(self, dates, valueColumn, i, sigma):
        """ 
        For each date, calculate the value
        """
        # initialize
        value_sum = 0
        value_weight = 0
        x = dates[i]
        l = len(dates)

        # compute value where there is nan 
        if math.isnan(valueColumn[i]):
            for k in range(l):
                value_single, weight = self.compute_single_value(dates, valueColumn, x, k, sigma)
                value_sum = value_sum + value_single
                value_weight = value_weight + weight

        # ignore values alread there
        else: 
            value_sum = valueColumn[i]
            value_weight = 1
        
        return value_sum / value_weight


    def compute_single_value(self, dates, valueColumn, x, k, sigma):
        """ 
        For each date, calculate the weight of ith value to it 
        """
        # extract date and values
        m_x = dates[k]
        m_v = valueColumn[k]

        if math.isnan(m_v):
            # ignore nan value
            m_v = 0
            weight = 0
            value_single = m_v * weight

        else: 
            # compute weight and weighted value
            weight = np.exp(-((m_x-x)**2)/(2*sigma**2))
            value_single = m_v * weight
        
        return value_single, weight 

#### 2.1.3 Fill the missing value in the df_covid

In [7]:
# create a container
df_covid_filled = pd.DataFrame()
df_covid_filled['location'] = df_covid['location']

# loop over columns to fill the missing value 
for (colname, colval) in df_covid.iloc[:, 1:].iteritems():
    list_filled = Datafilling(list(df_covid.index.values), colval.values.tolist(), 'float').filled_column
    df_covid_filled[colname] = list_filled

# display dataframe
df_covid_filled.head(5)


,location,new_cases,new_cases_per_million,new_deaths,new_deaths_per_million
date,,,,,
2020-02-27,Netherlands,1.000,0.057,66.125,0.052
2020-02-28,Netherlands,5.000,0.285,66.628,0.051
2020-02-29,Netherlands,4.000,0.228,67.123,0.050
2020-03-01,Netherlands,8.000,0.455,67.611,0.048
2020-03-02,Netherlands,6.000,0.342,68.090,0.047


#### 2.1.4 Smooth the covid data with Savitzky-Golay filter

In [8]:
# use Savitzky-Golay filter to smooth the eliminate the noices in the covid data

# create a container
df_covid_smoothed = pd.DataFrame()
df_covid_smoothed['location'] = df_covid_filled['location']

# loop over columns
for (columnName, columnData) in df_covid_filled.iloc[:, 1:].iteritems():
    df_covid_smoothed[columnName] = savgol_filter(columnData, 25, 4)

# check smothing quality
fig = px.line(df_covid_smoothed, y='new_cases', color='location', title='Daily new cases after smoothing')
fig.show()

#### 2.1.5 Export the processed data

In [9]:
df_covid_smoothed.to_csv('./data/travelling modes/processed/CovidData.csv')

### 2.2 Process transportation data

#### 2.2.1 Interpret the encoded data in the Netherlands transportation data
- As is diplayed in data import section, values in the columns of "Travle Modes", "Trip Characteristics", "Region", and "Year" are encoded, which need to be interpreted.
- Firstly, interpret the encoded data by the key-value pairs stored in file "NL_MODES".
- Secondly, format values in the column "Trip Characteristics".

In [10]:
# Import meta data to interpret values in df_transportNL
file_path = './data/travelling modes/raw/NL_MODES.csv' 
df_meta = pd.read_csv(file_path, delimiter=';')

# Interpret code in each cell in df_transportNL by KEY-VALUES paris in df_meta
replaceDict = pd.Series(df_meta.VALUES.values, index=df_meta.KEY)
df_transportNL['Trip Characteristics'] = df_transportNL['Trip Characteristics'].apply(str)
df_transportNL = df_transportNL.replace(replaceDict)

# display dataframe
df_transportNL_decoded = df_transportNL.set_index('Travel Modes', inplace=False)
df_transportNL_decoded.head(5)

,Trip Characteristics,Region,Year,Average Trip Per Day,Average Trip Per Year
Travel Modes,,,,,
Total,Sunday,The Netherlands,2018,2.06,107.0
Total,Sunday,The Netherlands,2019,1.94,101.0
Total,Sunday,The Netherlands,2020,1.64,85.0
Total,Sunday,The Netherlands,2021,1.83,95.0
Total,Sunday,Noord-Nederland (LD),2018,1.97,103.0


#### 2.2.2 Export transportation data

In [11]:
# export interpreted dataframe  
df_transportNL.to_csv('./data/travelling modes/processed/TransModes-NL.csv', index=False)
df_transportPL.to_csv('./data/travelling modes/processed/TransModes-PL.csv', index=False)

#### 2.2.3 Import processed Netherlands transportation data

The cell below categorizes the Netherlands' transportation modes into four types and calculates the percentage of each mode. It then creates a new dataframe to store the percentage values, with years from 2018 to 2022 as columns and different modes as indexes.

In [12]:
file_path = './data/travelling modes/processed/TransModes-NL.csv'
TransModes_NL = pd.read_csv(file_path, delimiter=',')
TransModes_NL =  TransModes_NL.reset_index(drop = True)

In [13]:
# categorize the transport modes and calculate their percentage in each year
def cal_percentage(dfTrans, string):
       ''' 
       Assume input is a dataframe and a year string,
       return a list contains percentage of each catagorized transportation mode in the given year.
       '''
       # extract rows of the netherlands nation level
       df = dfTrans[dfTrans['Region'] == 'The Netherlands']

       # drop unwanted transportation modes
       df = df[(df['Travel Modes'] != 'Total') & (df['Travel Modes'] != 'Passenger car (driver)') 
       & (df['Travel Modes'] != 'Walking')]

       # extract rows catagorized by week notation
       df = df[df['Trip Characteristics'].isin(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'])]

       # extract data of each year
       if string == '2018':
              df = df.query('Year==2018')
       elif string == '2019':
              df = df.query('Year==2019')
       elif string == '2020':
              df = df.query('Year==2020')
       elif string == '2021':
              df = df.query('Year==2021')       
       else:
              print('Invalid Year! Please Re-enter')

       # aggregate similar transportation modes
       df = df.drop_duplicates(subset=['Travel Modes', 'Trip Characteristics'], keep='first')
       df = df.groupby('Travel Modes').sum()
       df.loc['Railway transport'] = df.loc[['Bus/metro', 'Train']].sum(axis=0)
       df = df.drop(df.index[[1,4]])
       df = df.rename({'Passenger car (passenger)':'Road transport'}, axis=0)

       # calculate percentage and do formatting
       df['Percentage'] = (df['Average Trip Per Day']/df['Average Trip Per Day'].sum())*100
       df['Percentage'] = df['Percentage'].map('{:,.2f}'.format)

       return df['Percentage']

# create a new datafrmae
mode_Nl = pd.DataFrame() 

# call defiened function
mode_Nl['Percentage_2018'] = cal_percentage(TransModes_NL, '2018')
mode_Nl['Percentage_2019'] = cal_percentage(TransModes_NL, '2019')  
mode_Nl['Percentage_2020'] = cal_percentage(TransModes_NL, '2020')
mode_Nl['Percentage_2021'] = cal_percentage(TransModes_NL, '2021')

# display diffenrent transportation modes and their proportions in each year
mode_Nl.head(5)

,Percentage_2018,Percentage_2019,Percentage_2020,Percentage_2021
Travel Modes,,,,
Bike,57.32,56.68,61.14,59.47
Other,7.84,8.45,8.91,9.73
Road transport,23.40,23.42,23.26,24.13
Railway transport,11.44,11.44,6.69,6.67


#### 2.2.4 Import processed Poland transportation data

In [14]:
# import data
file_path = './data/travelling modes/processed/TransModes-PL.csv'
transModes_Pl = pd.read_csv(file_path, delimiter=',')
transModes_Pl.set_index('Travel Modes', inplace=True)

# aggregate similar transportation modes
transModes_Pl.loc['Other'] = transModes_Pl.loc[['Air transport', 'Inland waterway transport', 'Maritime transport']].sum(axis=0)
transModes_Pl = transModes_Pl.drop(transModes_Pl.index[[0, 3, 4, 5]]) 
transModes_Pl = transModes_Pl.drop(['Region'],axis=1)
transModes_Pl.loc['Bike'] = 0
'''transModes_Pl.loc['Road transport'] = transModes_Pl.loc['Road transport  ']
transModes_Pl = transModes_Pl.drop(['Road transport  '])'''

# display data
transModes_Pl.head(5)

,percentage_2021,percentage_2020,percentage_2019,percentage_2018
Travel Modes,,,,
Railway transport,48.8,55.8,57.9,46.7
Road transport,47.6,42.7,39.9,50.8
Other,3.6,1.5,2.2,2.5
Bike,0.0,0.0,0.0,0.0


#### 2.2.5 Integrate the two countries' transportation modes into a single dataframe.

In [15]:
# merge the percentage columns of each year in the two dataframes into a list
mode_percentage = pd.concat([
       mode_Nl['Percentage_2018'], 
       mode_Nl['Percentage_2019'], 
       mode_Nl['Percentage_2020'], 
       mode_Nl['Percentage_2021'],
       transModes_Pl['percentage_2018'], 
       transModes_Pl['percentage_2019'], 
       transModes_Pl['percentage_2020'], 
       transModes_Pl['percentage_2021']
])

# assign the list as a column of the new dataframe
mode_percentage = pd.DataFrame(data = mode_percentage)
mode_percentage['Percentage'] = mode_percentage.iloc[:, 0]
mode_percentage = mode_percentage.drop(columns = mode_percentage.columns[0])

# create columns of years and countries
mode_percentage['Year'] = (['2018'] * 4 + ['2019'] * 4 + ['2020'] * 4 + ['2021'] * 4) * 2
mode_percentage['Country'] = ['The Netherlands'] * 16 + ['Poland'] * 16
mode_percentage = mode_percentage.reset_index()

# display data
mode_percentage.head(5)

,Travel Modes,Percentage,Year,Country
0,Bike,57.32,2018,The Netherlands
1,Other,7.84,2018,The Netherlands
2,Road transport,23.40,2018,The Netherlands
3,Railway transport,11.44,2018,The Netherlands
4,Bike,56.68,2019,The Netherlands


In [16]:
mode_percentage.to_csv('./data/travelling modes/processed/mode_percentage.csv', index=False)